# EDA dataset Amazon's Top 50 bestselling books from 2009 to 2019

In [ ]:
# lod libraries
## Load libraries
import numpy as np
import pandas as pd
# import chart_studio.plotly as py
import cufflinks as cf
import  seaborn as sns
import plotly.express as px
%matplotlib inline

#  to work offline in local computer
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# load data

df = pd.read_csv("/kaggle/input/amazon-top-50-bestselling-books-2009-2019/bestsellers with categories.csv")
df.head()

# EDA

In [ ]:
# look at the shape of the data
df.shape

In [ ]:
# get info on the data
df.info()

* We have numerical and categorical variables

In [ ]:
# look if we have duplicated rows
df.duplicated().sum()

* No duplicated rows

In [ ]:
df['Author'].duplicated().sum() 

* 302 similar authors names in the data

In [ ]:
df['Name'].duplicated().sum()

* There is 199 identical books in the data

In [ ]:
# look at authors and book names duplicated

# look at how many duplicate authors/books names we have in the list?

def duplicate_cols(col_name):
    for c in col_name:
        if df[c].duplicated().any() ==True:
            print(f'Columns {c} has {df[c].duplicated().sum()} duplicates.')
        else:
            print(f'Columns {c} has no duplicates')


cols = ['Name', 'Author']
duplicate_cols(cols)

In [ ]:
# Check if spelling errors in Names and Authors names

for col in cols:
    print(f'Before {col}: {len(set(df[col]))} After {col}: {len(set(df[col].str.title().str.strip()))}')

* we hae typing errors in the names

In [ ]:
# Fix typing error in the name

df.Name = df['Name'].str.title().str.strip()


In [ ]:
# Check if the cahnge was included

for col in cols:
    print(f'Before {col}: {len(set(df[col]))} After {col}: {len(set(df[col].str.title().str.strip()))}')

In [ ]:
# Check the same author name spelling errors

A = df['Author'].sort_values().unique()
A



In [ ]:
# Fix name of authors with too much space in it
# exple":

# Let's build a list of the most similar spellings
import difflib
for name in A:
    l = difflib.get_close_matches(name, A)  # Most closer name to name
    if len(l) >1:
        print(l)

In [ ]:
# I saw only those two names that are similar
#.... This part need to be more automate so that we do not have to look manually at names

# ['George R. R. Martin', 'George R.R. Martin']
# ['J. K. Rowling', 'J.K. Rowling']

# Replace those name by the same name
# Replace the names of the authors with the correct ones
df = df.replace('George R. R. Martin', 'George R.R. Martin')
df = df.replace('J. K. Rowling', 'J.K. Rowling')


In [ ]:
# Take the data with same authors/books name

df_SameAuthor = df.loc[df['Author'].duplicated()]
# df_SameName = df.loc[df['N'].duplicated()]
df_SameAuthor.shape

In [ ]:
# Count the number of each book in the data

count_books = dict()

for k in list(df['Author']):
    count_books[k] = count_books.get(k, 0) + 1
# print(count_books)

# Which is the Author with more books in the data?
# list of 5 Authors

lst = list()
for key, val in count_books.items():
    newtup = (val, key)
    lst.append(newtup)

# sort the list descendent way
lst = sorted(lst, reverse=True)

# Take the 5 most popular books Authors in the data
for val, key in lst[:5]:
    print(key, val)
    

* The Author with the most books is "Jeff Kinney"

In [ ]:
# Extract all the books writen by this author
df_Jeff_Kinney = df.loc[df['Author'] =='Jeff Kinney']
df_Jeff_Kinney 

* We notice that 'Jeff mKinney' write only 'Fiction' books, and have very good User rating from 4.7
* He has been writing since 2009, and have wrote a book every year.
* His books are really not expenive

In [ ]:
# The Authors with the most expensive books

df_bestsale_author = df.loc[df['Price'] == df['Price'].max()]
df_bestsale_author 

*  For two years consecutive, the Author ' American Psychiatric Association' have sale his book very high price.

In [ ]:
# The Authors with the less expensive book

df_cheapsale_author = df.loc[df['Price'] == df['Price'].min()]
df_cheapsale_author 

In [ ]:
fig = px.scatter(df_cheapsale_author,
          x ='User Rating', 
          y = 'Author', 
          color ='Genre', 
          size= 'User Rating', 
           hover_data=['User Rating'],
           text='Year',
          )

fig.update_traces(textposition='top center')

fig.update_layout(title_text='Books with lower price')

fig.show()


In [ ]:
print(df_cheapsale_author.Year.max(), df_cheapsale_author.Year.min())

In [ ]:
df_cheapsale_author['User Rating'].max()

* The author Jeff Kinney have user rating progressing positively from 2010 to 2017
* Harper Lee have also a very good progressing User Rating
* Alice Schertle , Delegates of the Constitutional and Stephenie Meyer have books in 2010,2014 and 2016 with lower rate despide the lower price of the book.
*  RH Disney has a constant User rating of his book 

In [ ]:
# Create a stacked bar plot

px.bar(df,
       x = 'Year',
      y= 'Price',
      color ='Genre',
      title ='Price for each book genre on Each Year',
      labels ={'x': 'Prices', 'y': 'Years'})


* We clearly notice that the genre ' Non fiction" is more expensive than the genre 'Fiction'.
* We notice an price increase of the two genre until 2014 where the prices start to drop 
* in 2015, there is a massive drop of the Fiction genre of book ( the price is at his lower cost.

* The author whose book is/(has been) the more expensive ia 'E.L. James' in 2012, genre 'Fiction'

* The cheapest book was '10-Day Green Smoothie Cleanse' a non Fiction book written by 'JJ Smith'

In [ ]:
# Look at reviews per book genre
fig = px.bar(df,
             y ='Reviews', 
             x ='Genre',
#              text ='',
             color ='Genre',
             title ='Reviews of books per genre'
            )
fig

* The 'Fiction' genre has better reviews than the on fiction type of book 

In [ ]:
# Look in 2015 only where the price were the lowest for the two genres of book
df_2015 = df.loc[df['Year']==2015]


fig = px.bar(df_2015,
             y ='Reviews', 
             x ='Author',
             text ='User Rating',
#              barmode ='group',
             color ='Genre'
            )
# we add update the figure
fig.update_traces(texttemplate ='%{text:.2s}', textposition = 'outside')
fig.update_layout(uniformtext_minsize =12)
# rotate labels x to 45 degree
fig.update_layout(xaxis_tickangle =-45)

fig

* In 2015, the 'Fiction' books have the best reviews  but not the more expensive price.
*  The high number of reviews does not mean best user rating. 

In [ ]:
df_2015_best_price = df_2015.loc[df_2015['Price'] == df_2015['Price'].max()]
df_2015_best_price

In [ ]:
px.pie(df_2015, 
       values ='Reviews', 
       names ='Genre',
       title ='Reviews percentages per book type in 2015',
      color_discrete_sequence = px.colors.sequential.RdBu
       )


In [ ]:
px.histogram(df_2015, 
            x ='User Rating',
            color ='Genre',
            barmode='stack')


* High user rate for Fiction book in 2015

In [ ]:
df_Rating_best = df.loc[df['User Rating'] >= 4.9]
len(df_Rating_best['User Rating'])

# only 52 books have 4.9 and above rating score



In [ ]:
# plot the best rating books
px.bar(df_Rating_best,
       x = 'Year', 
       y = 'Reviews',
        color='Genre',
       barmode='group'
      )

In [ ]:
# Authors with the best User Rating
df_bestrate = df.loc[df['User Rating'] == df['User Rating'].max()]
df_bestrate = df_bestrate[['Author', 'User Rating']]


df_bestrate['Author'].unique()
# # Take only unique author names mand have a list of authors with best user rating 
# drop duplicated Author names
df_bestrate = df_bestrate.drop_duplicates(subset = 'Author', keep = 'first')
df_bestrate

### Conclusion:

*  The following authors have highest rating: ['Bill Martin Jr.',  'Dav Pilkey',  'Sherri Duskey Rinker',  'Lin-Manuel Miranda',  'J.K. Rowling',  'Brandon Stanton',
 'Sarah Young',  'Jill Twiss',  'Alice Schertle',  'Pete Souza',  'Dr. Seuss',  'Rush Limbaugh',  'Nathan W. Pyle',  'Patrick Thorpe',  'Chip Gaines',  'Eric Carle',
 'Emily Winfield Martin',  'Mark R. Levin',  'Jeff Kinney'].  They all have User Rating of 4.9.
 
 
* The 5 top best sellers authors are : 
 Jeff Kinney: 12
Suzanne Collins: 11
Rick Riordan: 11
Gary Chapman: 11
American Psychological Association: 10


* Author that write   'Non-fiction' book Genre is more likely to become a bestseller. 


  
    